Start with the standard imports we have used for every notebook in this class.

In [3]:
%matplotlib inline
import numpy as np
import scipy as sp
import matplotlib as mpl
import matplotlib.cm as cm
import matplotlib.pyplot as plt
import pandas as pd
pd.set_option('display.width', 500)
pd.set_option('display.max_columns', 100)
pd.set_option('display.notebook_repr_html', True)
import seaborn as sns
sns.set_style("whitegrid")
sns.set_context("poster")

#These are one time load of SCHOOL DISTRICT data steps.
#START SECTION OF DO NOT RUN THESE AGAIN

Each of the datasheets downloaded from ELSI had download metadata on the top of them and total and key information on the bottom of them that were not data rows. This metadata, total, and key information was manually deleted before import. Some of the files had ="0" instead of 0 in the cells. This was found/replaced before import using the sed -i '' 's/="0"/0/g' *.csv command from the terminal.

In [41]:
#CITATION: This is the data from National Center for Education Statistics on Schools
#School districts for all 50 states and Washington, D.C.
#http://nces.ed.gov/ccd/elsi/
#Data Source: U.S. Department of Education National Center for Education Statistics Common Core of Data (CCD) "Local Education Agency (School District) Universe Survey" 2009-10 v.2a  2013-14 v.1a; "Public Elementary/Secondary School Universe Survey" 2009-10 v.2a; "Survey of Local Government Finances School Systems (F-33)" 2009-10 (FY 2010) v.1a.
#KEY:
#† indicates that the data are not applicable.
#– indicates that the data are missing.
#‡ indicates that the data do not meet NCES data quality standards.

districtinformation = pd.read_csv("tempdata/school districts/2009-2010 DISTRICTS Information Tab.csv", dtype=np.str)
districtcharacteristicsa = pd.read_csv("tempdata/school districts/2009-2010 DISTRICTS CharacteristicsA Tab.csv", dtype=np.str)
districtcharacteristicsb = pd.read_csv("tempdata/school districts/2009-2010 DISTRICTS CharacteristicsB Tab.csv", dtype=np.str)
districtenrollments = pd.read_csv("tempdata/school districts/2009-2010 DISTRICTS Enrollments Tab.csv", dtype=np.str)
districtenrollmentK3 = pd.read_csv("tempdata/school districts/2009-2010 DISTRICTS EnrollK3 Tab.csv", dtype=np.str)
districtenrollment48 = pd.read_csv("tempdata/school districts/2009-2010 DISTRICTS Enroll48 Tab.csv", dtype=np.str)
districtenrollment912 = pd.read_csv("tempdata/school districts/2009-2010 DISTRICTS Enroll912 Tab.csv", dtype=np.str)
districtteacherstaff = pd.read_csv("tempdata/school districts/2009-2010 DISTRICTS TeacherStaff Tab.csv", dtype=np.str)
districtgeneralfinance = pd.read_csv("tempdata/school districts/2009-2010 DISTRICTS GeneralFinance Tab.csv", dtype=np.str)
districtrevenue = pd.read_csv("tempdata/school districts/2009-2010 DISTRICTS Revenue Tab.csv", dtype=np.str)
districtexpenditures = pd.read_csv("tempdata/school districts/2009-2010 DISTRICTS Expenditures Tab.csv", dtype=np.str)

#Data Source: Local Education Agency (School District) Universe Survey Dropout and Completion Data: 2009-10 v.1a.

districtdropoutscompleters = pd.read_csv("tempdata/school districts/2009-2010 DISTRICTS DropoutsCompleters.txt", dtype=np.str, delim_whitespace=True)

Check the lengths of the datasets to see if we have a row for every school district.

In [27]:
print len(districtinformation)
print len(districtcharacteristicsa)
print len(districtcharacteristicsb)
print len(districtenrollments)
print len(districtenrollmentK3)
print len(districtenrollment48)
print len(districtenrollment912)
print len(districtteacherstaff)
print len(districtgeneralfinance)
print len(districtrevenue)
print len(districtexpenditures)
print len(districtdropoutscompleters)

19023
17916
17916
17916
17916
17916
17916
17916
17916
17916
17916
18439


Drop all of the duplicate columns.

In [42]:
districtcharacteristicsb = districtcharacteristicsb.drop(districtcharacteristicsb.columns[[0, 1]], 1)
districtenrollments = districtenrollments.drop(districtenrollments.columns[[0, 1]], 1)
districtenrollmentK3 = districtenrollmentK3.drop(districtenrollmentK3.columns[[0, 1]], 1)
districtenrollment48 = districtenrollment48.drop(districtenrollment48.columns[[0, 1]], 1)
districtenrollment912 = districtenrollment912.drop(districtenrollment912.columns[[0, 1]], 1)
districtteacherstaff = districtteacherstaff.drop(districtteacherstaff.columns[[0, 1]], 1)
districtgeneralfinance = districtgeneralfinance.drop(districtgeneralfinance.columns[[0, 1]], 1)
districtrevenue = districtrevenue.drop(districtrevenue.columns[[0, 1]], 1)
districtexpenditures = districtexpenditures.drop(districtexpenditures.columns[[0, 1]], 1)

Join all of the school district datasets. The datasets districtinformation and districtdropoutscompleters need special treatment, as they have more rows for the school districts than the other datasets. All of the other datasets can be joined by ID without issue.

In [85]:
#Join the datasets that can be joined without issue.
joineddistrict = districtcharacteristicsa.join([districtcharacteristicsb, districtenrollments, districtenrollmentK3, districtenrollment48, districtenrollment912, districtteacherstaff, districtgeneralfinance, districtrevenue, districtexpenditures])

#Clean up an extra hidden character in the Agency Name column
joineddistrict = joineddistrict.rename(columns={'﻿Agency Name': 'Agency Name'})
districtinformation = districtinformation.rename(columns={'﻿Agency Name': 'Agency Name'})

#Merge to the districtinformation dataset
joineddistrict = districtinformation.merge(joineddistrict, 'outer', ['Agency Name', u'State Name [District] Latest available year'], suffixes=('', '_DEL'))

#Need to get rid of Excel syntax ="" from some of the columns
joineddistrict['Agency ID - NCES Assigned [District] Latest available year'] = joineddistrict['Agency ID - NCES Assigned [District] Latest available year'].map(lambda x: str(x).lstrip('="').rstrip('"'))
joineddistrict['County Number [District] 2009-10'] = joineddistrict['County Number [District] 2009-10'].map(lambda x: str(x).lstrip('="').rstrip('"'))
joineddistrict['ANSI/FIPS State Code [District] Latest available year'] = joineddistrict['ANSI/FIPS State Code [District] Latest available year'].map(lambda x: str(x).lstrip('="').rstrip('"'))
joineddistrict['Location ZIP [District] 2013-14'] = joineddistrict['Location ZIP [District] 2013-14'].map(lambda x: str(x).lstrip('="').rstrip('"'))
joineddistrict['Location ZIP4 [District] 2013-14'] = joineddistrict['Location ZIP4 [District] 2013-14'].map(lambda x: str(x).lstrip('="').rstrip('"'))
joineddistrict['Mailing ZIP [District] 2013-14'] = joineddistrict['Mailing ZIP [District] 2013-14'].map(lambda x: str(x).lstrip('="').rstrip('"'))
joineddistrict['Mailing ZIP4 [District] 2013-14'] = joineddistrict['Mailing ZIP4 [District] 2013-14'].map(lambda x: str(x).lstrip('="').rstrip('"'))
joineddistrict['School District Level Code (SCHLEV) [District Finance] 2009-10'] = joineddistrict['School District Level Code (SCHLEV) [District Finance] 2009-10'].map(lambda x: str(x).lstrip('="').rstrip('"'))
joineddistrict['Supervisory Union (ID) Number [District] 2009-10'] = joineddistrict['Supervisory Union (ID) Number [District] 2009-10'].map(lambda x: str(x).lstrip('="').rstrip('"'))
joineddistrict['Metro Micro Area Code [District] 2009-10'] = joineddistrict['Metro Micro Area Code [District] 2009-10'].map(lambda x: str(x).lstrip('="').rstrip('"'))
joineddistrict['Congressional Code [District] 2009-10'] = joineddistrict['Congressional Code [District] 2009-10'].map(lambda x: str(x).lstrip('="').rstrip('"'))
#joineddistrict[''] = joineddistrict[''].map(lambda x: x.lstrip('="').rstrip('"'))

#Rename the LEAID column so it can be merged with the joineddistrict dataset
districtdropoutscompleters = districtdropoutscompleters.rename(columns={'LEAID': 'Agency ID - NCES Assigned [District] Latest available year'})

#Merge to the joineddistrict dataset
joineddistrict = joineddistrict.merge(districtdropoutscompleters, 'outer', 'Agency ID - NCES Assigned [District] Latest available year', suffixes=('', '_DEL'))

#If by chance any rows have NaN, replace with the ELSI standard for missing data '–'
joineddistrict = joineddistrict.fillna('–')
joineddistrict = joineddistrict.replace('nan', '–')

#Where these conditions are true, these rows are not usable
joineddistrict = joineddistrict[joineddistrict['Agency Name']!='–']
joineddistrict = joineddistrict[joineddistrict['Agency Type [District] 2009-10']!='–']

joineddistrict.head()

,Agency Name,State Name [District] Latest available year,State Name [District] 2009-10,State Abbr [District] Latest available year,Agency Name [District] 2009-10,Agency ID - NCES Assigned [District] Latest available year,County Name [District] 2009-10,County Number [District] 2009-10,Race/Ethnicity Category [District] 2009-10,ANSI/FIPS State Code [District] Latest available year,Total Number Operational Schools [Public School] 2009-10,Total Number Operational Charter Schools [Public School] 2009-10,Total Number of Public Schools [Public School] 2009-10,Years District Reported Data [District] Latest available year,Years District Did Not Report Data [District] Latest available year,Location Address [District] 2013-14,Location City [District] 2013-14,Location State Abbr [District] 2013-14,Location ZIP [District] 2013-14,Location ZIP4 [District] 2013-14,Mailing Address [District] 2013-14,Mailing City [District] 2013-14,Mailing State Abbr [District] 2013-14,Mailing ZIP [District] 2013-14,Mailing ZIP4 [District] 2013-14,Phone Number [District] 2013-14,Agency Type [District] 2009-10,School District Level Code (SCHLEV) [District Finance] 2009-10,Urban-centric Locale [District] 2009-10,Boundary Change Indicator Flag [District] 2009-10,CBSA Name [District] 2009-10,CBSA ID [District] 2009-10,CSA Name [District] 2009-10,CSA ID [District] 2009-10,Latitude [District] 2009-10,Longitude [District] 2009-10,State Agency ID [District] 2009-10,Supervisory Union (ID) Number [District] 2009-10,Agency Charter Status [District] 2009-10,Metro Micro Area Code [District] 2009-10,Congressional Code [District] 2009-10,Census ID (CENSUSID) [District Finance] 2009-10,Lowest Grade Offered [District] 2009-10,Highest Grade Offered [District] 2009-10,Total Students (UG PK-12) [District] 2009-10,PK thru 12th Students [District] 2009-10,Ungraded Students [District] 2009-10,Total Students [Public School] 2009-10,Limited English Proficient (LEP) / English Language Learners (ELL) [District] 2009-10,Individualized Education Program Students [District] 2009-10,...,Total - School Admin.- Supp. Serv. Exp. (E09) [District Finance] 2009-10,Total - Student Transp.- Supp. Serv. Exp. (V45) [District Finance] 2009-10,Total - Students- Supp. Serv. Exp. (E17) [District Finance] 2009-10,Salary - Instruction Expenditures (Z33) [District Finance] 2009-10,Salary - Students- Supp. Serv. Exp. (V11) [District Finance] 2009-10,Salary - Instruct. Staff- Supp. Serv. Exp. (V13) [District Finance] 2009-10,Salary - General Admin.- Supp. Serv. Exp. (V15) [District Finance] 2009-10,Salary - School Admin.- Supp. Serv. Exp. (V17) [District Finance] 2009-10,Salary - Ops. & Mainten.- Supp. Serv. Exp. (V21) [District Finance] 2009-10,Salary - Student Transp.- Supp. Serv. Exp. (V23) [District Finance] 2009-10,Salary - Other Supp. Serv.- Supp. Serv. Exp. (V37) [District Finance] 2009-10,Salary - Food Services- Non-Instruction (V29) [District Finance] 2009-10,Employee Benefits - Instruction Expend. (V10) [District Finance] 2009-10,Empl. Benefits - Students- Supp. Serv. Exp. (V12) [District Finance] 2009-10,Empl. Benefits - Instruction- Supp. Serv. Exp. (V14) [District Finance] 2009-10,Empl. Benefits - Gen. Adm.- Supp. Serv. Exp. (V16) [District Finance] 2009-10,Empl. Benefits - Sch. Adm.- Supp. Serv. Exp. (V18) [District Finance] 2009-10,Empl. Benefits - Ops. & Maint.- Supp. Serv. Exp. (V22) [District Finance] 2009-10,Empl. Benefits - Student Trans.- Supp. Serv. Exp. (V24) [District Finance] 2009-10,Empl. Benefits - Other Supp Serv- Supp. Serv. Exp. (V38) [District Finance] 2009-10,Empl. Benefits - Food Services- Non-Instruction (V30) [District Finance] 2009-10,Empl. Benefits - Enterp. Oper.- Non-Instruction (V32) [District Finance] 2009-10,Current Spending - Private Schools (V91) [District Finance] 2009-10,Current Spending - Public Charter Schools (V92) [District Finance] 2009-10,Teacher Salaries - Regular Education Programs (Z35) [District Finance] 2009-10,Teacher Salaries - Special Education Progr

In [86]:
print len(joineddistrict)

18184


In [87]:
joineddistrict.to_csv("tempdata/districts.csv")

#END SECTION OF DO NOT RUN THESE AGAIN
#START Exploratory Data Analysis

In [88]:
districtsdf = pd.read_csv("tempdata/districts.csv")
districtsdf.head()

/Users/MacAce/anaconda/lib/python2.7/site-packages/pandas/io/parsers.py:1170: DtypeWarning: Columns (8,11,35,36,41,463,464,465,466,467,468,469,470) have mixed types. Specify dtype option on import or set low_memory=False.
  data = self._reader.read(nrows)


,Unnamed: 0,Agency Name,State Name [District] Latest available year,State Name [District] 2009-10,State Abbr [District] Latest available year,Agency Name [District] 2009-10,Agency ID - NCES Assigned [District] Latest available year,County Name [District] 2009-10,County Number [District] 2009-10,Race/Ethnicity Category [District] 2009-10,ANSI/FIPS State Code [District] Latest available year,Total Number Operational Schools [Public School] 2009-10,Total Number Operational Charter Schools [Public School] 2009-10,Total Number of Public Schools [Public School] 2009-10,Years District Reported Data [District] Latest available year,Years District Did Not Report Data [District] Latest available year,Location Address [District] 2013-14,Location City [District] 2013-14,Location State Abbr [District] 2013-14,Location ZIP [District] 2013-14,Location ZIP4 [District] 2013-14,Mailing Address [District] 2013-14,Mailing City [District] 2013-14,Mailing State Abbr [District] 2013-14,Mailing ZIP [District] 2013-14,Mailing ZIP4 [District] 2013-14,Phone Number [District] 2013-14,Agency Type [District] 2009-10,School District Level Code (SCHLEV) [District Finance] 2009-10,Urban-centric Locale [District] 2009-10,Boundary Change Indicator Flag [District] 2009-10,CBSA Name [District] 2009-10,CBSA ID [District] 2009-10,CSA Name [District] 2009-10,CSA ID [District] 2009-10,Latitude [District] 2009-10,Longitude [District] 2009-10,State Agency ID [District] 2009-10,Supervisory Union (ID) Number [District] 2009-10,Agency Charter Status [District] 2009-10,Metro Micro Area Code [District] 2009-10,Congressional Code [District] 2009-10,Census ID (CENSUSID) [District Finance] 2009-10,Lowest Grade Offered [District] 2009-10,Highest Grade Offered [District] 2009-10,Total Students (UG PK-12) [District] 2009-10,PK thru 12th Students [District] 2009-10,Ungraded Students [District] 2009-10,Total Students [Public School] 2009-10,Limited English Proficient (LEP) / English Language Learners (ELL) [District] 2009-10,...,Total - School Admin.- Supp. Serv. Exp. (E09) [District Finance] 2009-10,Total - Student Transp.- Supp. Serv. Exp. (V45) [District Finance] 2009-10,Total - Students- Supp. Serv. Exp. (E17) [District Finance] 2009-10,Salary - Instruction Expenditures (Z33) [District Finance] 2009-10,Salary - Students- Supp. Serv. Exp. (V11) [District Finance] 2009-10,Salary - Instruct. Staff- Supp. Serv. Exp. (V13) [District Finance] 2009-10,Salary - General Admin.- Supp. Serv. Exp. (V15) [District Finance] 2009-10,Salary - School Admin.- Supp. Serv. Exp. (V17) [District Finance] 2009-10,Salary - Ops. & Mainten.- Supp. Serv. Exp. (V21) [District Finance] 2009-10,Salary - Student Transp.- Supp. Serv. Exp. (V23) [District Finance] 2009-10,Salary - Other Supp. Serv.- Supp. Serv. Exp. (V37) [District Finance] 2009-10,Salary - Food Services- Non-Instruction (V29) [District Finance] 2009-10,Employee Benefits - Instruction Expend. (V10) [District Finance] 2009-10,Empl. Benefits - Students- Supp. Serv. Exp. (V12) [District Finance] 2009-10,Empl. Benefits - Instruction- Supp. Serv. Exp. (V14) [District Finance] 2009-10,Empl. Benefits - Gen. Adm.- Supp. Serv. Exp. (V16) [District Finance] 2009-10,Empl. Benefits - Sch. Adm.- Supp. Serv. Exp. (V18) [District Finance] 2009-10,Empl. Benefits - Ops. & Maint.- Supp. Serv. Exp. (V22) [District Finance] 2009-10,Empl. Benefits - Student Trans.- Supp. Serv. Exp. (V24) [District Finance] 2009-10,Empl. Benefits - Other Supp Serv- Supp. Serv. Exp. (V38) [District Finance] 2009-10,Empl. Benefits - Food Services- Non-Instruction (V30) [District Finance] 2009-10,Empl. Benefits - Enterp. Oper.- Non-Instruction (V32) [District Finance] 2009-10,Current Spending - Private Schools (V91) [District Finance] 2009-10,Current Spending - Public Charter Schools (V92) [District Finance] 2009-10,Teacher Salaries - Regular Education Programs (Z35) [District Finance] 2009-10,Teacher Salaries - Special Education Programs (Z36) [District Finance] 2009-10,Teacher Salar